<a href="https://colab.research.google.com/github/iakioh/MusiCAN/blob/main/models/first_music_GAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import torch

## Settings

In [6]:
# Pianoroll parameters
notes_per_octave = 12
number_octaves = 6   # as in MuseGAN
number_pitches = number_octaves * notes_per_octave

number_bars = 1
beats_per_bar = 4   # 4/4 rythm
blips_per_beat = 24   # temporal resolution of Lakh dataset
number_blips  = number_bars * beats_per_bar * blips_per_beat

print(f"pianoroll array: {number_pitches}x{number_blips}")


# Generator parameters
seed_length = 64

pianoroll array: 72x96


## Data Preparation

## Model

In [ ]:
class Generator (torch.nn.Module) :
    """
    first GAN generator
    """

    def __init__ (self) : 
        super().__init__()
        self.layer0 = torch.nn.Linear(seed_length, 2*seed_length)
